In [1]:
# Copyright 2024 Ethan Lee Christensen
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import re
import os
import openai
from bruhcolor import bruhcolored as bc

from dotenv import load_dotenv
load_dotenv()

# client = openai.OpenAI()
client = openai.AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

In [2]:
def complete(system_prompt: str, user_prompt: str, model: str = os.getenv("MODEL")) -> str:
    try:
        return client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=1500,
            model=model
        ).choices[0].message.content
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [3]:
complete(
    "helpful friend",
    "Hello there",
    os.getenv("MODEL")
)

'Hi! How can I help you today?'

In [4]:
def generate_outline(story_type: str, target_audience: str):
    system_prompt = """
You are an expert story outliner.
You will be provided a type of story and target audience.
Your task is to take these two items and create a story outline that has 5 main plot points.

You will be given the story type surrounded in four hashtags (####). As an example, #### <story type> ####.    
You will be given the target audience surrounded in four dollar signs ($$$$). As an example, $$$$ <story type> $$$$.

Rules:
- You MUST generate a rich story outline with 3 to 5 main plot points.
- You MUST lead main points with a single dash.
- You MUST provide 5 to 10 sub points for each main point that explains the story line for the main point.
- You MUST lead sub points with two dashes.


Example Input:
Story Type:
####
<story type here>
####

Target Audience:
$$$$
<target audience here>
$$$$


Example Output:
- Point one
-- sub point 1-1
-- sub point 1-2
-- sub point 1-3
. . . 
"""

    user_prompt = """
Story Type:
####
${story_type}
####

Target Audience:
$$$$
${target_audience}
$$$$
"""

    response = complete(
        system_prompt=system_prompt,
        user_prompt=user_prompt.replace("${story_type}", story_type).replace("${target_audience}", target_audience)
    ).split("\n")

    response = [val for val in response if val.strip() != ""]

    chapter_points = {}

    current_chapter = ""

    for val in response:
        if re.match(r"\-\s.*", val):
            current_chapter = val.split("- ")[1]
            chapter_points[current_chapter] = []
        elif re.match(r"\-\-\s.*", val):
            chapter_points[current_chapter].append(val.split("-- ")[1])
        else:
            print("ERROR")

    return chapter_points

In [5]:
def chapter_page_generator(
    story_type: str,
    target_audience: str,
    chapter_title: str,
    chapter_point: str,
    previous_knowledge: str,
    completed_chapters: str,
    last_paragraph: str, 
    chapter_save_path: str,
    page_number: int,
    last_chapter: bool = False,
    last_page: bool = False
) -> str:
    system_prompt = """
You are a expert partial story generator.
You will be provided a series of details regarding a story.
Your task is to take this information and return a single, multi paragraph page to add to the story.

Information That will be provided to you:
- The story title / purpose surrounded by four hashtags: #### <story title / purpose> ####
- The target audience surrounded by four @ symbols: @@@@ <target audience> @@@@
- The title of the current chapter you are generating surrounded by four dollar signs: $$$$ <chapter title> $$$$
- The quick overview of the page you need to generate surrounded in four ateriks: **** <page to create> ****
- Optionally: previous knowledge of the previous page to keep you on track surrounded in four exclamation points: !!!! <previous knowledge> !!!!
- If it is the last chapter, you will be told so.
- If it is the last page to generate, you will be told so.

Rules / Task:
- Use the provided information to create the next page in the story.
- Only provide the page content and nothing else.
"""

    user_prompt = """
Story title:
####
${story_title}
####

Target Audience:
@@@@
${target_audience}
@@@@

Chapter Title:
$$$$
${chapter_title}
$$$$

Previous Knowledge that may be helpful:
!!!!
    Previously Completed Pages for the current Chapter:
${previous_knowledge}

    Previously Completed Chapters:
${completed_chapters}

    Last Paragraph from the Previous Chapter:
${last_paragraph}

This ${last_chapter} the last chapter of the book.
This ${last_page} the last page of the book.
!!!!

Given the information above, create me page content that is about this following sentence:
Page to Create Overview:
****
${page_to_create}
**** 
"""

    if previous_knowledge == "":
        previous_knowledge = "No Previous Knowledge."
    if completed_chapters == "":
        completed_chapters = "No Previously Completed Chapters."
    if last_paragraph == "":
        last_paragraph = "No Last Paragraph - Start of a new chatper."

    filled_user_prompt = (
        user_prompt.replace("${story_title}", story_type)
        .replace("${target_audience}", target_audience)
        .replace("${chapter_title}", chapter_title)
        .replace("${page_to_create}", chapter_point)
        .replace("${previous_knowledge}", previous_knowledge)
        .replace("${completed_chapters}", completed_chapters)
        .replace("${last_paragraph}", last_paragraph)
        .replace("${last_chapter}", "is" if last_chapter else "is not")
        .replace("${last_page}", "is" if last_page else "is not")
    )

    new_page = complete(system_prompt=system_prompt, user_prompt=filled_user_prompt)
    
    last_paragraph = [val for val in new_page.split("\n") if val.strip() != ""][-1]

    page_path = f"{chapter_save_path}/page_{page_number}.txt"

    with open(page_path, "w") as file:
        print(bc(f"Creating Chapter Page: '{page_path}'", color=87))
        file.write(
            f"Story Type: {story_type}\n"
            + f"Chapter Title: {chapter_title}\n\n"
            + new_page
            + "\n"
        )

    return (
        previous_knowledge
        + f"Completed Page: {chapter_title} - Page {page_number} - {chapter_point}\n", last_paragraph
    )

In [6]:
def story_generator(story_type: str, target_audience: str, story_outline: dict[str, list[str]]) -> None:
    
    cleaned_story_type = re.sub(r"[^A-Za-z0-9 ]", "", story_type).replace(" ", "_")
    
    if not os.path.exists("./stories"):
        print(bc("Creating 'stories' base folder", color=82))
        os.mkdir("./stories")
    
    if not os.path.exists(f"./stories/{cleaned_story_type}"):
        print(bc(f"Creating 'stories/{cleaned_story_type}' story folder", color=72))
        os.mkdir(f"./stories/{cleaned_story_type}")
    
    base_path = f"./stories/{cleaned_story_type}"
    
    chapter_number = 0
    
    completed_chapters = ""
    
    for chapter_num, chapter_val in enumerate(story_outline.items()):
        last_chapter = chapter_num == len(story_outline) - 1
        chapter, chapter_points = chapter_val
        chapter_number += 1
        previous_chapter_knowledge = ""
        last_paragraph = ""
        cleaned_chapter_name = f"{chapter_number}_" + re.sub(r"[^A-Za-z0-9 ]", "", chapter).replace(" ", "_")
        chapter_page_save_path = f"./stories/{cleaned_story_type}/{cleaned_chapter_name}"
        print(bc(f"Creating '{chapter_page_save_path}'", color=132))
        os.mkdir(chapter_page_save_path)
        for idx, point in enumerate(chapter_points):
            last_page = idx == len(chapter_points) - 1 
            previous_chapter_knowledge, last_paragraph = chapter_page_generator(
                story_type=story_type,
                target_audience=target_audience,
                chapter_title=chapter,
                chapter_point=point,
                previous_knowledge=previous_chapter_knowledge,
                completed_chapters=completed_chapters,
                last_paragraph=last_paragraph,
                chapter_save_path=chapter_page_save_path,
                page_number=idx,
                last_chapter=last_chapter,
                last_page=last_page
            )
        completed_chapters += f"Previously Completed Chapter Title - {chapter}\n"

In [7]:
story_type = "An epic adventure of a goldfish named Finn."
target_audience = "Children aged 5-10"

story_outline = generate_outline(
    story_type=story_type,
    target_audience=target_audience
)

story_generator(
    story_type=story_type,
    target_audience=target_audience,
    story_outline=story_outline
)

Creating 'stories/An_epic_adventure_of_a_goldfish_named_Finn' story folder
Creating './stories/An_epic_adventure_of_a_goldfish_named_Finn/1_Finn_a_goldfish_living_in_a_small_fish_tank_dreams_of_exploring_the_world_beyond_his_tank'
Creating Chapter Page: './stories/An_epic_adventure_of_a_goldfish_named_Finn/1_Finn_a_goldfish_living_in_a_small_fish_tank_dreams_of_exploring_the_world_beyond_his_tank/page_0.txt'
Creating Chapter Page: './stories/An_epic_adventure_of_a_goldfish_named_Finn/1_Finn_a_goldfish_living_in_a_small_fish_tank_dreams_of_exploring_the_world_beyond_his_tank/page_1.txt'
Creating Chapter Page: './stories/An_epic_adventure_of_a_goldfish_named_Finn/1_Finn_a_goldfish_living_in_a_small_fish_tank_dreams_of_exploring_the_world_beyond_his_tank/page_2.txt'
Creating Chapter Page: './stories/An_epic_adventure_of_a_goldfish_named_Finn/1_Finn_a_goldfish_living_in_a_small_fish_tank_dreams_of_exploring_the_world_beyond_his_tank/page_3.txt'
Creating './stories/An_epic_adventure_of_a_go